## 1. Import Libraries

In [7]:
import pandas as pd
import numpy as np
from nocasedict import NocaseDict
import requests
import math
import simplejson

## 2. Establish Key Variables

In [8]:
# The direct reference to the LGBF file on the IS website could not be used this time around as there were a number of errors that required manual correction. Open the Error Outputs folder to see details.
LGBF_DataFile_Url = "C:\\Users\\lowsona\\Desktop\\2020-21 LGBF data file.xlsx"

## 3. Establish Functions

In [9]:
# Establish function to unpivot data as this will be repeated for each sheet. This function only works with the LGBF data file because the layout of the 4 sheets is the same.
def unpivotData (df) :
    # Promote Headers
    new_headers = df.iloc[0]
    df = df[1:]
    df.columns = new_headers
    # Unpivot Data - !!!! This is creating additional blank rows for num/den dataframes - look into fix. Currently not causing any real issues (as far as I can tell from basic checks of the resulting csv) as additional blank rows can just be dropped!!!!
    df = df.melt(id_vars = ['GSS Code', 'Local Authority'],var_name = 'Attribute',value_name = 'Value')
    # For some reason only the indicators coded as corp contain a space between the code text and the number. Added a step here to correct it as it applies to all dataframes and causes an issue with splitting code and period from the single string format it is supplied in due to inconsistency.
    df['Attribute'] = df['Attribute'].str.replace('CORP ','CORP')
    return df

# Establish function that provides a signed difference between two values.
def distance(Current, Previous): 
    return (max(Previous, Current) - min(Previous, Current)) * (-1 if Previous > Current else 1)



## 4. Establish Key Dataframes
Get each sheet from the excel file and load them into their own distinct dataframes. Basic transformation steps will be applied here to ensure each dataframe has appropriate headers, data is unpivoted etc. More in depth transformation will be handled later.

### 4.1 Establish Dataframes from LGBF Raw Data XLSX File

In [10]:
# Collect the excel file from the url/location provided
dataFile = pd.ExcelFile(LGBF_DataFile_Url)
# Write each sheet within the file to it's own data frame. These will eventually be combined into a single more usable dataframe. When rerunning this notebook be careful of the sheet names the Improvement Service have chosen (in particular trailing spaces).
cashIndicators = pd.read_excel(dataFile,'Cash Indicators')
cashIndicators_NumeratorDenominator = pd.read_excel(dataFile,' Cash Num-Den_Indi')
realIndicators = pd.read_excel(dataFile,'Real Indicators')
realIndicators_NumeratorDenominator = pd.read_excel(dataFile,'Real Num_Den_Indi')

# Output realIndicators to give an example of the output from this step.
realIndicators

,Unnamed: 0,All costs are real values,CHN1 - Cost per primary school pupil,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 913,Unnamed: 914,Unnamed: 915,Unnamed: 916,Unnamed: 917,Unnamed: 918,Unnamed: 919,Unnamed: 920,Unnamed: 921,Unnamed: 922
0,GSS Code,Local Authority,CHN1 2010-11 Real,CHN1 2011-12 Real,CHN1 2012-13 Real,CHN1 2013-14 Real,CHN1 2014-15 Real,CHN1 2015-16 Real,CHN1 2016-17 Real,CHN1 2017-18 Real,...,CLIM2 2011-12,CLIM2 2012-13,CLIM2 2013-14,CLIM2 2014-15,CLIM2 2015-16,CLIM2 2016-17,CLIM2 2017-18,CLIM2 2018-19,CLIM2 2019-20,CLIM2 2020-21
1,S12000033,Aberdeen City,6193.57298,6002.861299,5978.152403,5736.885414,6078.710053,5873.505905,5615.977165,5747.913919,...,6.827374,7.121462,6.695704,5.924096,5.665047,5.221048,5.023961,4.91221,4.772118,4.156801
2,S12000034,Aberdeenshire,6050.771574,5627.812273,5634.272979,5689.719627,5615.466543,5876.473046,5848.695448,5808.147749,...,7.206827,7.462463,7.21763,6.71164,6.443418,6.290225,6.154452,5.940581,5.739133,5.171514
3,S12000041,Angus,5838.823993,5741.207793,5504.130032,5349.642714,5467.679121,5501.764557,5671.248151,5657.452781,...,6.715932,7.064489,6.830112,6.213756,6.041542,5.672699,5.568539,5.471535,5.143943,4.635357
4,S12000035,Argyll & Bute,7302.747865,6918.786873,6960.547134,6881.861221,6595.573083,6476.061104,6542.322208,7374.481001,...,7.282303,7.870511,7.382207,6.78237,6.589629,6.192544,6.095812,5.943149,5.601903,4.944015
5,S12000005,Clackmannanshire,5521.957869,5370.825565,4922.81457,4584.743146,4519.064732,4647.753028,5110.825321,5402.6689,...,8.999268,8.333544,8.270623,7.077736,7.623643,7.541096,7.170596,7.207906,6.855824,6.237931
6,S12000006,Dumfries & Galloway,7193.155829,7150.229871,6727.493762,6641.094988,6477.904806,6489.795709,6340.9917,6292.947957,...,8.189104,8.529028,8.384926,7.575013,7.449139,7.234891,7.029954,6.786626,6.440854,5.829122
7,S12000042,Dundee City,5908.457423,5599.527636,5279.748191,5107.303851,4967.568166,4840.842861,5034.785815,5299.954668,...,6.092007,6.249366,5.984092,5.277946,5.134059,4.740296,4.494786,4.358474,4.047518,3.928813
8,S12000008,East Ayrshire,5846.435957,5405.067581,5493.646351,5379.527273,5307.327125,5630.890556,5807.110151,5786.775765,...,6.071271,6.226184,6.176965,5.559674,5.417011,5.105743,5.009466,4.900941,4.699207,4.169993
9,S12000045,East Dunbartonshire,6017.782096,5852.225563,5658.63166,5399.843902,5381.020271,5223.59712,5411.041784,5235.199895,...,4.936222,4.915226,4.772582,4.290605,4.185856,3.89785,3.774416,3.660129,3.517073,3.15581


In [11]:
# Unpivot all other dataframes so that they are in a more sensible starting point. Each dataframe will require it's own specific transformations so this is as far as the generic steps applied to all dataframes can go.
cashIndicators = unpivotData(cashIndicators)
realIndicators = unpivotData(realIndicators)
cashIndicators_NumeratorDenominator = unpivotData(cashIndicators_NumeratorDenominator)
realIndicators_NumeratorDenominator = unpivotData(realIndicators_NumeratorDenominator)

# Output realIndicators to give an example of the output from this step.
realIndicators


,GSS Code,Local Authority,Attribute,Value
0,S12000033,Aberdeen City,CHN1 2010-11 Real,6193.57298
1,S12000034,Aberdeenshire,CHN1 2010-11 Real,6050.771574
2,S12000041,Angus,CHN1 2010-11 Real,5838.823993
3,S12000035,Argyll & Bute,CHN1 2010-11 Real,7302.747865
4,S12000005,Clackmannanshire,CHN1 2010-11 Real,5521.957869
...,...,...,...,...
30388,S12000029,South Lanarkshire,CLIM2 2020-21,3.338181
30389,S12000030,Stirling,CLIM2 2020-21,5.349171
30390,S12000039,West Dunbartonshire,CLIM2 2020-21,3.611371
30391,S12000040,West Lothian,CLIM2 2020-21,3.854523


### 4.2 Establish Indicator Information Dataframe
To refresh this from the SharePoint list use "Get Indicator Info.ipynb"

In [12]:
indicatorInformation = pd.read_csv("Data Files//Indicator Information.csv")
indicatorInformation

,Title,Code,Code_Sortable,ReportingPeriod,MeasureType,NumberFormat,YMin,YMax,ISCategory,Committee,...,Ranking_GoldilocksMidpoint,NumberFormat_Axis,Format_Python,FormatAxis_Python,AdditionalAxisDenominator_Python,FormatAxis_Plotly_Prefix,FormatAxis_Plotly_Suffix,ImgPxlWidth_Plotly,ImgPxlHeight_Plotly,SubGroup_PythonReport
0,Net cost per Waste collection per premises,ENV1a,ENV 01a,Annual,Cost,'£ '0,40.0,NaN,Environmental Services,Environment and Housing,...,NaN,'£ '0,£ {:0.0f},",d",1,£,NaN,812,305,Waste & Recycling
1,Net cost per Waste disposal per premises,ENV2a,ENV 02a,Annual,Cost,'£ '0,60.0,NaN,Environmental Services,Environment and Housing,...,NaN,'£ '0,£ {:0.0f},",d",1,£,NaN,812,305,Waste & Recycling
2,The % of total household waste arising that is...,ENV6,ENV 06,Annual,Percentage,0.0%,40.0,100.0,Environmental Services,Environment and Housing,...,NaN,0%,{:.1%},",.0%",1,NaN,NaN,812,305,Waste & Recycling
3,% of adults satisfied with refuse collection,ENV7a,ENV 07a,3 Year Aggregates,Percentage,0.0%,50.0,100.0,Environmental Services,Environment and Housing,...,NaN,0%,{:.1%},",.0%",1,NaN,NaN,812,305,Waste & Recycling
4,"Net cost of street cleaning per 1,000 population",ENV3a,ENV 03a,Annual,Cost,"'£ ' #,###",10000.0,NaN,Environmental Services,Environment and Housing,...,NaN,"'£ ' #,##0,""K""",£ {:.2f}K,s,1000,£,NaN,800,493,Street Cleaning
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,% of council dwellings that are energy efficient,HSN5,HSN 05,Annual,Percentage,0.0%,NaN,NaN,Housing Services,Environment and Housing,...,NaN,0.0%,{:.1%},.1%,1,NaN,NaN,812,305,Housing - Property and Assets
98,% of Children Living in Poverty (After Housing...,CHN24,CHN 24,Annual,Percentage,0.0%,NaN,NaN,Children's Services,Children and Young People,...,NaN,NaN,{:.1%},NaN,1,NaN,NaN,812,305,Children & Families Social Work
99,Gross Value Added (GVA) per capita,ECON11,ECON 11,Annual,Cost,"'£ ' #,###",NaN,NaN,Economic Development,Finance and Economy,...,NaN,NaN,£ {:.2f}K,s,1,£,NaN,812,305,Economic Growth
100,Claimant Count as a % of Working Age Population,ECON12a,ECON 12a,Annual,Percentage,0.0%,NaN,NaN,Economic Development,Finance and Economy,...,NaN,0%,{:.1%},",.0%",1,NaN,NaN,812,305,Employment


### 4.2 Establish Family Group Dataframe

In [13]:
Family_Groups = pd.read_csv("Data Files//Family Groups.csv")
Family_Groups

,Local_Authority,Type,Family_Group
0,Eilean Siar,"Environmental, Culture & Leisure, Economic Dev...",Family Group 1
1,Argyll & Bute,"Environmental, Culture & Leisure, Economic Dev...",Family Group 1
2,Shetland Islands,"Environmental, Culture & Leisure, Economic Dev...",Family Group 1
3,Highland,"Environmental, Culture & Leisure, Economic Dev...",Family Group 1
4,Orkney Islands,"Environmental, Culture & Leisure, Economic Dev...",Family Group 1
...,...,...,...
59,North Ayrshire,"Children, Social Work and Housing indicators",Family Group 4
60,North Lanarkshire,"Children, Social Work and Housing indicators",Family Group 4
61,Inverclyde,"Children, Social Work and Housing indicators",Family Group 4
62,West Dunbartonshire,"Children, Social Work and Housing indicators",Family Group 4


## 5. Transform Each Dataframe Containing Indicator Data and Merge DataFrames Into One Master DataFrame
Apply relevant transformation steps to each of the Dataframes below and join them into a single fact table containing Cash Values, Real Values, Cash Numerators, Cash Denominators, Real Numerators and Real Denominators.

### 5.1 Transform realIndicators Dataframe

In [14]:
#Import the realIndicators into what will become the final table
all_LGBFData = realIndicators
#Split the Attribute column into its parts using space as a delimeter
all_LGBFData[['Code','Period','Other']] = all_LGBFData['Attribute'].str.split(" ", n=2, expand=True)
#Remove the Attribute column and the redundant other column, rename columns to avoid spaces and to identify the value type as we intend to merge all dataframes into a single dataframe.
all_LGBFData = all_LGBFData.drop(columns = ['Attribute','Other'])
all_LGBFData = all_LGBFData.rename(columns = {'Value': 'Real_Value', 'Local Authority' : 'Local_Authority'})
# Save Scotland values only for merge with averages table later
ISScottishValues_Real = all_LGBFData[(all_LGBFData.Local_Authority == "Scotland")]
#Remove all of the "Scottish Average" Rows as we intend to provide both types of average later in order to provide a more "Complete" data set. Reset the index after this to ensure it is consistent.
all_LGBFData = all_LGBFData[(all_LGBFData.Local_Authority != "Scotland")]
all_LGBFData = all_LGBFData.reset_index(drop=True)
all_LGBFData

,GSS Code,Local_Authority,Real_Value,Code,Period
0,S12000033,Aberdeen City,6193.57298,CHN1,2010-11
1,S12000034,Aberdeenshire,6050.771574,CHN1,2010-11
2,S12000041,Angus,5838.823993,CHN1,2010-11
3,S12000035,Argyll & Bute,7302.747865,CHN1,2010-11
4,S12000005,Clackmannanshire,5521.957869,CHN1,2010-11
...,...,...,...,...,...
29467,S12000028,South Ayrshire,4.925712,CLIM2,2020-21
29468,S12000029,South Lanarkshire,3.338181,CLIM2,2020-21
29469,S12000030,Stirling,5.349171,CLIM2,2020-21
29470,S12000039,West Dunbartonshire,3.611371,CLIM2,2020-21


### 5.2 Transform cashIndicators Dataframe

In [15]:
#Split the Attribute column into its parts using space as a delimeter
cashIndicators[['Code','Period']] = cashIndicators['Attribute'].str.split(" ", n=2, expand=True)
#Remove the Attribute column and the redundant other column, rename columns to avoid spaces and to identify the value type as we intend merge into a single dataframe.
cashIndicators = cashIndicators.drop(columns = ['Attribute'])
cashIndicators = cashIndicators.rename(columns = {'Value': 'Cash_Value', 'Local Authority' : 'Local_Authority'})
#Merge with saved scotland real values to merge with Scottish Averages Dataframe later
ISScottishValues_Cash = cashIndicators[(cashIndicators.Local_Authority == "Scotland")]
ISScottishValues = ISScottishValues_Real.merge(ISScottishValues_Cash, how = 'left', on = ['Local_Authority','Code','Period','GSS Code'], suffixes = ('_Real','_Cash'))
ISScottishValues = ISScottishValues.drop(columns = ['GSS Code','Local_Authority'])
#Remove all of the "Scottish Average" Rows as we intend to provide both types of average later in order to provide a more "Complete" data set. Reset the index after this to ensure it is consistent.
cashIndicators = cashIndicators[(cashIndicators.Local_Authority != "Scotland")]
cashIndicators = cashIndicators.reset_index(drop=True)
cashIndicators

,GSS Code,Local_Authority,Cash_Value,Code,Period
0,S12000033,Aberdeen City,4960.336134,CHN1,2010-11
1,S12000034,Aberdeenshire,4845.968712,CHN1,2010-11
2,S12000041,Angus,4676.223196,CHN1,2010-11
3,S12000035,Argyll & Bute,5848.657025,CHN1,2010-11
4,S12000005,Clackmannanshire,4422.450053,CHN1,2010-11
...,...,...,...,...,...
29467,S12000028,South Ayrshire,4.925712,CLIM2,2020-21
29468,S12000029,South Lanarkshire,3.338181,CLIM2,2020-21
29469,S12000030,Stirling,5.349171,CLIM2,2020-21
29470,S12000039,West Dunbartonshire,3.611371,CLIM2,2020-21


### 5.4 Transform NumeratorDenominator Dataframes

In [16]:
# Create a dictionary that can be searched independant of case from the indicatorInformation dataframe. This avoids having to loop the dataframe and exponentially speeds this step up.
indicatorInformation_dict = indicatorInformation.set_index('Code').to_dict('index')
indicatorInformation_dict = NocaseDict(indicatorInformation_dict)
# Display example dictionary item for reference
indicatorInformation_dict['env1a']

{'Title': 'Net cost per Waste collection per premises',
 'Code_Sortable': 'ENV 01a',
 'ReportingPeriod': 'Annual',
 'MeasureType': 'Cost',
 'NumberFormat': "'£ '0",
 'YMin': 40.0,
 'YMax': nan,
 'ISCategory': 'Environmental Services',
 'Committee': 'Environment and Housing',
 'FamilyGrouping': 'Environmental, Culture & Leisure, Economic Development, Corporate and Property indicators',
 'StirlingService': 'Environment & Place ; Waste Services',
 'Ranking_Type': 'Ascending',
 'NumberFormat_NoText': 0.0,
 'Source': 'For more details on Net Waste Collection Costs see LFRs Guidance notes for waste collection services:\nhttp://www.scotland.gov.uk/Topics/Statistics/Browse/Local-Government-Finance/ReturnLFR/LFR1112Guidance (LFR 06, Waste Collection)\nData for Number of Premises can be found on the Audit Scotland website:\nhttp://www.audit-scotland.gov.uk/performance/service/ Waste Management\nFrom 2013/14 Councils return the number of premises data direct to the Improvement Service. Data is av

In [17]:
# Define function to convert numerator/denominator sheets into column based dataframes that only contain the necessary columns

Types = []
Codes = []
Periods = []
Values = []
PreviousType = None
PreviousAttribute = None

def ConvertNumDenDF(df,prefix) :
    #!!! This is a bit hacky - Requires proper fix in unpivot data table!!!
    df = df[pd.notnull(df['Attribute'])]
    df = df.rename(columns = {'Local Authority' : 'Local_Authority'})

    # Filter the dataframe to remove Scotland values as they will be calculated later.
    df = df[(df.Local_Authority != "Scotland")]

    #Get first word from Attribute column (this will be the code where the value specified is not a numerator or denominator)
    df['Code'] = df['Attribute'].str.split(' ').str[0]
    df['Year'] = df['Attribute'].str.split(' ').str[1]

    df['Value'] = df['Value'].apply(pd.to_numeric,errors='coerce')

    #Loop dataframe and establish Value Type, Period and Code based on positioning and checking the code column in the indicatorInformation dataframe. This will have issues if new indicators are introduced and the codes have not yet been added to the Indicator Information SharePoint List. Manual checking required as not sure how to protect against this.
    Types = []
    Codes = []
    Periods = []
    Values = []
    PreviousType = None
    PreviousAttribute = None

    for row in df.itertuples():
        
        if row.Attribute == PreviousAttribute:
            Types.append(PreviousType)
            Codes.append(LastCode)
            Periods.append(LastPeriod)
            PreviousAttribute = row.Attribute

        elif any(row.Code in string for string in indicatorInformation['Code']):
            Types.append("Value")
            LastCode = row.Code
            Codes.append(LastCode)
            LastPeriod = row.Year
            Periods.append(LastPeriod)
            PreviousType = "Value"
            PreviousAttribute = row.Attribute
            Numerator_Multiplier = indicatorInformation_dict.get(row.Code).get('Numerator_Multipier')
            Denominator_Multiplier = indicatorInformation_dict.get(row.Code).get('Denominator_Multiplier')

        elif PreviousType == "Value" :
            Types.append("Numerator")
            Codes.append(LastCode)
            Periods.append(LastPeriod)
            PreviousType = "Numerator"
            PreviousAttribute = row.Attribute

        elif PreviousType == "Numerator":
            Types.append("Denominator")
            Codes.append(LastCode)
            Periods.append(LastPeriod)
            PreviousType = "Denominator"
            PreviousAttribute = row.Attribute

        else:
            Types.append(None)
            Codes.append(None)
            Periods.append(None)
            PreviousAttribute = row.Attribute

        if PreviousType == "Value":
            Values.append(row.Value)
        elif PreviousType == "Numerator":
            Values.append(row.Value * Numerator_Multiplier)
        elif PreviousType == "Denominator":
            Values.append(row.Value * Denominator_Multiplier)

    df['Code'] = Codes
    df['Period'] = Periods
    df['Type'] = Types
    df['Value'] = Values

    df = df[['GSS Code','Local_Authority','Code','Period','Type','Value']]
    df = df[pd.notnull(df['Value'])]

    # Establish new dataframe that contains only the values from realIndicators_NumeratorDenominator that are numerators. Rename the Value column appropriately
    df_Numerators = df[(df.Type == "Numerator")]
    df_Numerators = df_Numerators.rename(columns = {'Value' : prefix + '_Numerator'})
    df_Numerators = df_Numerators.drop('Type', axis=1)

    # Establish new dataframe that contains only the values from realIndicators_NumeratorDenominator that are denominators. Rename the Value column appropriately
    df_Denominators = df[(df.Type == "Denominator")]
    df_Denominators = df_Denominators.rename(columns={'Value': prefix + '_Denominator'})
    df_Denominators = df_Denominators.drop('Type', axis=1)

    # Merge realIndicators_Numerators and realIndicators_Denominators overwriting the previous dataframe realIndicators_NumeratorDenominator so we have a dataframe of the correct length with two value columns. One for numerator and one for denominator.
    df = df_Numerators.merge(df_Denominators, how='left', on=['Local_Authority', 'Code', 'Period', 'GSS Code'], suffixes=('_Num', '_Den'))
    return df

In [18]:
# Convert sheets and output one as an example
realIndicators_NumeratorDenominator = ConvertNumDenDF(realIndicators_NumeratorDenominator,'Real')
cashIndicators_NumeratorDenominator = ConvertNumDenDF(cashIndicators_NumeratorDenominator,'Cash')
cashIndicators_NumeratorDenominator

,GSS Code,Local_Authority,Code,Period,Cash_Numerator,Cash_Denominator
0,S12000033,Aberdeen City,CHN1,2010-11,59028000.0,11900.0
1,S12000034,Aberdeenshire,CHN1,2010-11,92621000.0,19113.0
2,S12000041,Angus,CHN1,2010-11,39472000.0,8441.0
3,S12000035,Argyll & Bute,CHN1,2010-11,33969000.0,5808.0
4,S12000005,Clackmannanshire,CHN1,2010-11,16823000.0,3804.0
...,...,...,...,...,...,...
13562,S12000028,South Ayrshire,FINSUS5,2020-21,272864000.0,297218000.0
13563,S12000029,South Lanarkshire,FINSUS5,2020-21,779667000.0,779667000.0
13564,S12000030,Stirling,FINSUS5,2020-21,221273000.0,224088000.0
13565,S12000039,West Dunbartonshire,FINSUS5,2020-21,242658000.0,242696000.0


### 5.5 Merge all Dataframes into all_LGBFData

In [19]:
# Merge the dataframes together into all_LGBFData. Reorder columns after merge.
all_LGBFData = all_LGBFData.merge(cashIndicators, how = 'left', on = ['Local_Authority','Code','Period','GSS Code'], suffixes = ('_Real','_Cash'))
all_LGBFData = all_LGBFData.merge(realIndicators_NumeratorDenominator, how = 'left', on = ['Local_Authority','Code','Period','GSS Code'], suffixes = ('_all','_realNumDen'))
all_LGBFData = all_LGBFData.merge(cashIndicators_NumeratorDenominator, how = 'left', on = ['Local_Authority','Code','Period','GSS Code'], suffixes = ('_all','_cashNumDen'))
all_LGBFData = all_LGBFData[['GSS Code','Local_Authority','Code','Period','Real_Value','Real_Numerator','Real_Denominator','Cash_Value','Cash_Numerator','Cash_Denominator']]

# Convert value columns to numeric only using errors = 'coerce' to force any text values (dna, *, N/A etc.) to return NaN.
all_LGBFData[['Real_Value','Cash_Value']] = all_LGBFData[['Real_Value','Cash_Value']].apply(pd.to_numeric,errors='coerce')

# Remove rows which contain null or NaN in the Real_Value column as they are non-datapoints and could interfere with average, ranking and quartile calculations later.
all_LGBFData = all_LGBFData[pd.notnull(all_LGBFData['Real_Value'])]

# Convert Percentages to proper decimal percentages
all_LGBFData = all_LGBFData.merge(indicatorInformation[['Code','MeasureType']], how = 'left', on = ['Code'], suffixes = ('_all','_info'))

def ConvertPercent_Real(df) : 
    if df['MeasureType'] == "Percentage":
        return df['Real_Value']/100
    else :
        return df['Real_Value']

def ConvertPercent_Cash(df) : 
    if df['MeasureType'] == "Percentage":
        return df['Cash_Value']/100
    else :
        return df['Cash_Value']

all_LGBFData['Real_Value'] = all_LGBFData.apply(ConvertPercent_Real, axis = 1)
all_LGBFData['Cash_Value'] = all_LGBFData.apply(ConvertPercent_Cash, axis = 1)

all_LGBFData = all_LGBFData.drop(columns = ['MeasureType'])

all_LGBFData

,GSS Code,Local_Authority,Code,Period,Real_Value,Real_Numerator,Real_Denominator,Cash_Value,Cash_Numerator,Cash_Denominator
0,S12000033,Aberdeen City,CHN1,2010-11,6193.572980,7.370352e+07,11900.0,4960.336134,59028000.0,11900.0
1,S12000034,Aberdeenshire,CHN1,2010-11,6050.771574,1.156484e+08,19113.0,4845.968712,92621000.0,19113.0
2,S12000041,Angus,CHN1,2010-11,5838.823993,4.928551e+07,8441.0,4676.223196,39472000.0,8441.0
3,S12000035,Argyll & Bute,CHN1,2010-11,7302.747865,4.241436e+07,5808.0,5848.657025,33969000.0,5808.0
4,S12000005,Clackmannanshire,CHN1,2010-11,5521.957869,2.100553e+07,3804.0,4422.450053,16823000.0,3804.0
...,...,...,...,...,...,...,...,...,...,...
28471,S12000028,South Ayrshire,CLIM2,2020-21,4.925712,NaN,NaN,4.925712,NaN,NaN
28472,S12000029,South Lanarkshire,CLIM2,2020-21,3.338181,NaN,NaN,3.338181,NaN,NaN
28473,S12000030,Stirling,CLIM2,2020-21,5.349171,NaN,NaN,5.349171,NaN,NaN
28474,S12000039,West Dunbartonshire,CLIM2,2020-21,3.611371,NaN,NaN,3.611371,NaN,NaN


## 6. Calculate Averages
There are two types of mean average calculations that will be applied to both real and cash values within both the Scottish 32 Councils whole group and the Family Groups for each indicator and period. In addition to these mean average calculations the median averages have also been added. All averages to be added are as below along with the column names to be assigned.

1. Scottish Averages
    1. Average of Local Authority Real Values - ScotAv_LA_Real
    2. Average of Local Authority Cash Values - ScotAv_LA_Cash
    3. Sum of Real Numerators Divided by Sum of Real Denominators - ScotAv_NumDen_Real
    4. Sum of Cash Numerators Divided by Sum of Cash Denominators - ScotAv_NumDen_Cash
    5. Average of Local Authority Real Numerator Values - ScotAv_LA_Num_Real
    6. Average of Local Authority Real Denominator Values - ScotAv_LA_Den_Real
    5. Average of Local Authority Cash Numerator Values - ScotAv_LA_Num_Cash
    6. Average of Local Authority Cash Denominator Values - ScotAv_LA_Den_Cash
2. Family Group Averages
    1.  Average of Local Authority Real Values - FamilyAv_LA_Real
    2.  Average of Local Authority Cash Values - FamilyAv_LA_Cash
    3.  Sum of Real Numerators Divided by Sum of Real Denominators - FamilyAv_NumDen_Real
    4.  Sum of Cash Numerators Divided by Sum of Cash Denominators - FamilyAv_NumDen_Cash
    5.  Average of Local Authority Real Numerator Values - FamilyAv_LA_Num_Real
    6.  Average of Local Authority Real Denominator Values - FamilyAv_LA_Den_Real
    7.  Average of Local Authority Cash Numerator Values - FamilyAv_LA_Num_Cash
    8.  Average of Local Authority Cash Denominator Values - FamilyAv_LA_Den_Cash
    9.  Median of Local Authority Real Values - FamilyMed_LA_Real
    10. Median of Local Authority Cash Values - FamilyMed_LA_Cash
    11. Median of Local Authority Real Numerator Values - FamilyMed_LA_Num_Real
    12. Median of Local Authority Real Denominator Values - FamilyMed_LA_Den_Real
    13. Median of Local Authority Cash Numerator Values - FamilyMed_LA_Num_Cash
    14. Median of Local Authority Cash Denominator Values - FamilyMed_LA_Den_Cash

### 6.1 Scottish Averages

In [20]:
#Sum then divide to get the average resulting from sum of numerator values divided by sum of denominator values for all Real Values.
real_NumeratorDenominatorAverages = realIndicators_NumeratorDenominator.groupby(['Code','Period'], as_index = False).sum()
real_NumeratorDenominatorAverages['ScotAv_NumDen_Real'] = real_NumeratorDenominatorAverages['Real_Numerator'] / real_NumeratorDenominatorAverages['Real_Denominator']
real_NumeratorDenominatorAverages = real_NumeratorDenominatorAverages[['Code','Period','ScotAv_NumDen_Real']]
real_NumeratorDenominatorAverages = real_NumeratorDenominatorAverages.replace([np.inf, -np.inf], np.nan)

#Sum then divide to get the average resulting from sum of numerator values divided by sum of denominator values for all Cash Values.
cash_NumeratorDenominatorAverages = cashIndicators_NumeratorDenominator.groupby(['Code','Period'], as_index = False).sum()
cash_NumeratorDenominatorAverages['ScotAv_NumDen_Cash'] = cash_NumeratorDenominatorAverages['Cash_Numerator'] / cash_NumeratorDenominatorAverages['Cash_Denominator']
cash_NumeratorDenominatorAverages = cash_NumeratorDenominatorAverages[['Code','Period','ScotAv_NumDen_Cash']]
cash_NumeratorDenominatorAverages = cash_NumeratorDenominatorAverages.replace([np.inf, -np.inf], np.nan)

#Calculate mean averages for all other columns and then merge dataframes together to finalise dataframe
ScottishAverages_LA = all_LGBFData.groupby(['Code','Period'], as_index = False).mean()
ScottishAverages =  ScottishAverages_LA.merge(real_NumeratorDenominatorAverages, how = 'left', on = ['Code','Period'], suffixes = ('_all','_RealNumDen'))
ScottishAverages =  ScottishAverages.merge(cash_NumeratorDenominatorAverages, how = 'left', on = ['Code','Period'], suffixes = ('_all','_CashNumDen'))

# Reorder and rename columns appropriately
ScottishAverages = ScottishAverages.rename(columns = {'Real_Value': 'ScotAv_LA_Real', 'Cash_Value' : 'ScotAv_LA_Cash', 'Real_Numerator' : 'ScotAv_LA_Num_Real', 'Real_Denominator' : 'ScotAv_LA_Den_Real','Cash_Numerator' : 'ScotAv_LA_Num_Cash', 'Cash_Denominator' : 'ScotAv_LA_Den_Cash'})

ScottishAverages = ScottishAverages.merge(indicatorInformation[['Code','Code_Sortable']], how = 'left', on = ['Code'], suffixes = ('_ScotAv','_info'))
ScottishAverages = ScottishAverages[['Code_Sortable','Period','ScotAv_LA_Real','ScotAv_LA_Num_Real','ScotAv_LA_Den_Real','ScotAv_NumDen_Real','ScotAv_LA_Cash','ScotAv_LA_Num_Cash','ScotAv_LA_Den_Cash','ScotAv_NumDen_Cash']]

ScottishAverages = ScottishAverages.rename(columns = {'Code_Sortable':'Code'})

# Add Relationship Key Column
ScottishAverages['Key_CodePeriod'] = ScottishAverages['Code'] + ScottishAverages['Period']

ScottishAverages

,Code,Period,ScotAv_LA_Real,ScotAv_LA_Num_Real,ScotAv_LA_Den_Real,ScotAv_NumDen_Real,ScotAv_LA_Cash,ScotAv_LA_Num_Cash,ScotAv_LA_Den_Cash,ScotAv_NumDen_Cash,Key_CodePeriod
0,C&L 01,2010-11,4.117713,6.299364e+06,1.420619e+06,4.434238,3.297813,5.045062e+06,1.420619e+06,3.551312,C&L 012010-11
1,C&L 01,2011-12,3.856182,5.977993e+06,1.506323e+06,3.968599,3.135006,4.860000e+06,1.506323e+06,3.226399,C&L 012011-12
2,C&L 01,2012-13,3.700006,6.104296e+06,1.613272e+06,3.783799,3.068692,5.062750e+06,1.613272e+06,3.138188,C&L 012012-13
3,C&L 01,2013-14,3.626466,6.264533e+06,1.666276e+06,3.759601,3.076593,5.314656e+06,1.666276e+06,3.189541,C&L 012013-14
4,C&L 01,2014-15,3.507325,5.687260e+06,1.647039e+06,3.453020,3.009798,4.880500e+06,1.647039e+06,2.963196,C&L 012014-15
...,...,...,...,...,...,...,...,...,...,...,...
903,SW 08,2016-17,794.060450,1.160719e+04,1.382216e+01,839.752300,794.060450,1.160719e+04,1.382216e+01,839.752300,SW 082016-17
904,SW 08,2017-18,708.904317,1.066372e+04,1.399909e+01,761.743506,708.904317,1.066372e+04,1.399909e+01,761.743506,SW 082017-18
905,SW 08,2018-19,737.560000,1.126409e+04,1.421050e+01,792.659917,737.560000,1.126409e+04,1.421050e+01,792.659917,SW 082018-19
906,SW 08,2019-20,710.784121,1.126447e+04,1.455766e+01,773.783125,710.784121,1.126447e+04,1.455766e+01,773.783125,SW 082019-20


### 6.2 Family Group Averages

In [21]:
#Merge all_LGBFData with indicatorInformation and Family_Groups to add the additional grouping information that enables Family Group average calculations.
FamilyGroupData = all_LGBFData.merge(indicatorInformation[['Code','FamilyGrouping']], how = 'left', on = ['Code'], suffixes = ('_all','_info'))
FamilyGroupData = FamilyGroupData.rename(columns = {'FamilyGrouping' : 'Type'})
FamilyGroupData = FamilyGroupData.merge(Family_Groups,how ='left', on = ['Local_Authority','Type'], suffixes = ('_all','_group'))

#Sum then divide to get the average resulting from sum of numerator values divided by sum of denominator values for all Cash Values.
real_NumeratorDenominatorAverages = FamilyGroupData.groupby(['Code','Period','Family_Group'], as_index = False).sum()
real_NumeratorDenominatorAverages['FamilyAv_NumDen_Real'] = real_NumeratorDenominatorAverages['Real_Numerator'] / real_NumeratorDenominatorAverages['Real_Denominator']
real_NumeratorDenominatorAverages = real_NumeratorDenominatorAverages[['Code','Period','Family_Group','FamilyAv_NumDen_Real']]
real_NumeratorDenominatorAverages = real_NumeratorDenominatorAverages.replace([np.inf, -np.inf], np.nan)

#Sum then divide to get the average resulting from sum of numerator values divided by sum of denominator values for all Cash Values.
cash_NumeratorDenominatorAverages = FamilyGroupData.groupby(['Code','Period','Family_Group'], as_index = False).sum()
cash_NumeratorDenominatorAverages['FamilyAv_NumDen_Cash'] = cash_NumeratorDenominatorAverages['Cash_Numerator'] / cash_NumeratorDenominatorAverages['Cash_Denominator']
cash_NumeratorDenominatorAverages = cash_NumeratorDenominatorAverages[['Code','Period','Family_Group','FamilyAv_NumDen_Cash']]
cash_NumeratorDenominatorAverages = cash_NumeratorDenominatorAverages.replace([np.inf, -np.inf], np.nan)

#Calculate mean and median averages and then merge dataframes together to finalise dataframe
FamilyAverages_LA = FamilyGroupData.groupby(['Code','Period','Family_Group'], as_index = False).mean()
FamilyMedians_LA = FamilyGroupData.groupby(['Code', 'Period', 'Family_Group'], as_index=False).median()
FamilyAverages =  FamilyAverages_LA.merge(FamilyMedians_LA, how='left', on=['Code', 'Period', 'Family_Group'], suffixes=('_Average', '_Median'))
FamilyAverages =  FamilyAverages.merge(real_NumeratorDenominatorAverages, how = 'left', on = ['Code','Period','Family_Group'], suffixes = ('_all','_RealNumDen'))
FamilyAverages =  FamilyAverages.merge(cash_NumeratorDenominatorAverages, how = 'left', on = ['Code','Period','Family_Group'], suffixes = ('_all','_CashNumDen'))

# Reorder and rename columns appropriately
FamilyAverages = FamilyAverages.rename(columns = {'Real_Value_Average': 'FamilyAv_LA_Real', 'Cash_Value_Average' : 'FamilyAv_LA_Cash', 'Real_Numerator_Average' : 'FamilyAv_LA_Num_Real', 'Real_Denominator_Average' : 'FamilyAv_LA_Den_Real','Cash_Numerator_Average' : 'FamilyAv_LA_Num_Cash', 'Cash_Denominator_Average' : 'FamilyAv_LA_Den_Cash', 'Real_Value_Median': 'FamilyMed_LA_Real', 'Cash_Value_Median' : 'FamilyMed_LA_Cash', 'Real_Numerator_Median' : 'FamilyMed_LA_Num_Real', 'Real_Denominator_Median' : 'FamilyMed_LA_Den_Real','Cash_Numerator_Median' : 'FamilyMed_LA_Num_Cash', 'Cash_Denominator_Median' : 'FamilyMed_LA_Den_Cash'})

FamilyAverages = FamilyAverages.merge(indicatorInformation[['Code','Code_Sortable']], how = 'left', on = ['Code'], suffixes = ('_FamilyAv','_info'))
FamilyAverages = FamilyAverages[['Code_Sortable','Period','Family_Group','FamilyAv_LA_Real','FamilyAv_LA_Num_Real','FamilyAv_LA_Den_Real','FamilyAv_NumDen_Real','FamilyAv_LA_Cash','FamilyAv_LA_Num_Cash','FamilyAv_LA_Den_Cash','FamilyAv_NumDen_Cash','FamilyMed_LA_Real','FamilyMed_LA_Cash','FamilyMed_LA_Num_Real','FamilyMed_LA_Den_Real','FamilyMed_LA_Num_Cash','FamilyMed_LA_Den_Cash']]

FamilyAverages = FamilyAverages.rename(columns = {'Code_Sortable':'Code'})

# Add Relationship Key Column
FamilyAverages['Key_CodePeriodFamily_Group'] = FamilyAverages['Code'] + FamilyAverages['Period'] + FamilyAverages['Family_Group']

FamilyAverages


,Code,Period,Family_Group,FamilyAv_LA_Real,FamilyAv_LA_Num_Real,FamilyAv_LA_Den_Real,FamilyAv_NumDen_Real,FamilyAv_LA_Cash,FamilyAv_LA_Num_Cash,FamilyAv_LA_Den_Cash,FamilyAv_NumDen_Cash,FamilyMed_LA_Real,FamilyMed_LA_Cash,FamilyMed_LA_Num_Real,FamilyMed_LA_Den_Real,FamilyMed_LA_Num_Cash,FamilyMed_LA_Den_Cash,Key_CodePeriodFamily_Group
0,C&L 01,2010-11,Family Group 1,3.759906,3.840910e+06,9.231241e+05,4.160773,3.011250,3076125.000,9.231241e+05,3.332298,4.051771,3.245000,2.441051e+06,6.954440e+05,1955000.0,6.954440e+05,C&L 012010-11Family Group 1
1,C&L 01,2010-11,Family Group 2,4.504395,5.208929e+06,1.165654e+06,4.468676,3.607500,4171750.000,1.165654e+06,3.578893,4.863373,3.895000,5.124335e+06,1.005080e+06,4104000.0,1.005080e+06,C&L 012010-11Family Group 2
2,C&L 01,2010-11,Family Group 3,3.864478,5.353457e+06,1.393280e+06,3.842339,3.095000,4287500.000,1.393280e+06,3.077270,4.151660,3.325000,4.296500e+06,1.336986e+06,3441000.0,1.336986e+06,C&L 012010-11Family Group 3
3,C&L 01,2010-11,Family Group 4,4.342075,1.079416e+07,2.200419e+06,4.905503,3.477500,8644875.000,2.200419e+06,3.928741,4.145417,3.320000,5.165539e+06,1.691615e+06,4137000.0,1.691615e+06,C&L 012010-11Family Group 4
4,C&L 01,2011-12,Family Group 1,3.738284,3.952579e+06,1.036476e+06,3.813478,3.039157,3213375.000,1.036476e+06,3.100289,3.523927,2.864889,2.245437e+06,7.075155e+05,1825500.0,7.075155e+05,C&L 012011-12Family Group 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3627,SW 08,2019-20,Family Group 4,735.254218,1.059762e+04,1.475225e+01,718.373468,735.254218,10597.625,1.475225e+01,718.373468,544.286153,544.286153,5.157500e+03,1.147850e+01,5157.5,1.147850e+01,SW 082019-20Family Group 4
3628,SW 08,2020-21,Family Group 1,283.731250,4.999500e+03,1.466362e+01,340.945707,283.731250,4999.500,1.466362e+01,340.945707,239.726650,239.726650,3.452000e+03,1.401650e+01,3452.0,1.401650e+01,SW 082020-21Family Group 1
3629,SW 08,2020-21,Family Group 2,472.165437,6.135125e+03,1.190350e+01,515.405133,472.165437,6135.125,1.190350e+01,515.405133,433.908400,433.908400,4.093000e+03,1.002650e+01,4093.0,1.002650e+01,SW 082020-21Family Group 2
3630,SW 08,2020-21,Family Group 3,530.394813,9.095125e+03,1.745237e+01,521.139673,530.394813,9095.125,1.745237e+01,521.139673,550.297900,550.297900,7.407000e+03,1.439200e+01,7407.0,1.439200e+01,SW 082020-21Family Group 3


## 7 Calculate Ranks & Percentiles

### 7.1 Scottish Ranks & Percentiles

In [22]:
#Copy all_LGBFData into ScottishRanks to avoid any steps affecting the all_LGBFData dataframe.
ScottishRanks = all_LGBFData.copy(deep=True)

#Add columns for both ascending and descending ranks and percentiles. The correct versions will be chosen later.
ScottishRanks['ScotRank_Desc'] = all_LGBFData.groupby(['Code','Period'])['Real_Value'].rank('min',ascending = False).astype(int)
ScottishRanks['ScotRank_Asc'] = all_LGBFData.groupby(['Code','Period'])['Real_Value'].rank('min',ascending = True).astype(int)
ScottishRanks['ScotRank_Desc_Pct'] = all_LGBFData.groupby(['Code','Period'])['Real_Value'].rank('min',ascending = False, pct = True).astype(float)
ScottishRanks['ScotRank_Asc_Pct'] = all_LGBFData.groupby(['Code', 'Period'])['Real_Value'].rank('min', ascending=True, pct=True).astype(float)
ScottishRanks = ScottishRanks.merge(indicatorInformation[['Code','Ranking_Type']], how = 'left', on = ['Code'], suffixes = ('_ScotAv','_Info'))


#Merge all_LGBFData with indicatorInformation to get the goldilocks mid points for the few indicators that use them.
GoldilocksScottishRanks = all_LGBFData.merge(indicatorInformation[['Code','Ranking_GoldilocksMidpoint']], how = 'left', on = ['Code'], suffixes = ('_Goldi','_Info'))

#Define function that returns unsigned difference between the real values and the goldilocks mid point. This uses the distance function established at the start of the notebook and then absolutes the values returned from it.
def DifferenceFromGoldilocksMidPoint(df) : 
    if df['Ranking_GoldilocksMidpoint'] == None :
        return None
    else :
        return abs(distance(df['Real_Value'],df['Ranking_GoldilocksMidpoint']))

#Calculate ranking and percentile based on distance from goldilocks mid point.
GoldilocksScottishRanks['AbsoluteDifferenceFromGoldilocksMidPoint'] = GoldilocksScottishRanks.apply(DifferenceFromGoldilocksMidPoint, axis = 1)
GoldilocksScottishRanks = GoldilocksScottishRanks[pd.notnull(GoldilocksScottishRanks['AbsoluteDifferenceFromGoldilocksMidPoint'])]
GoldilocksScottishRanks['ScotRank_Goldi'] = GoldilocksScottishRanks.groupby(['Code','Period'])['AbsoluteDifferenceFromGoldilocksMidPoint'].rank('min',ascending = True).astype(int)
GoldilocksScottishRanks['ScotRank_Goldi_Pct'] = GoldilocksScottishRanks.groupby(['Code','Period'])['AbsoluteDifferenceFromGoldilocksMidPoint'].rank('min',ascending = True,pct = True).astype(float)

#Merge goldilocks ranking and percentile into ScottishRanks dataframe
ScottishRanks = ScottishRanks.merge(GoldilocksScottishRanks[['Code','Period','Local_Authority','ScotRank_Goldi','ScotRank_Goldi_Pct']], how = 'left', on = ['Code','Period','Local_Authority'], suffixes = ('_ScotRank','_Goldi'))

# Define functions needed to select correct ranking type and percentile type
def ScotRank_select(df) : 
    if df['Ranking_Type'] == "Ascending" :
        return df['ScotRank_Asc']
    elif df['Ranking_Type'] == "Descending" :
        return df['ScotRank_Desc']
    elif df['Ranking_Type'] == "Goldilocks" :
        return df['ScotRank_Goldi']
    else :
        return None

def ScotRank_Pct_select(df) : 
    if df['Ranking_Type'] == "Ascending" :
        return df['ScotRank_Asc_Pct']
    elif df['Ranking_Type'] == "Descending" :
        return df['ScotRank_Desc_Pct']
    elif df['Ranking_Type'] == "Goldilocks" :
        return df['ScotRank_Goldi_Pct']
    else :
        return None

# Apply functions above to create two new columns that contain the correct rank and percentile for each row
ScottishRanks['ScotRank'] = ScottishRanks.apply(ScotRank_select, axis = 1)
ScottishRanks['ScotPct'] = ScottishRanks.apply(ScotRank_Pct_select, axis = 1)

# Add Scottish Ranks to the main fact table
all_LGBFData = ScottishRanks[['GSS Code','Local_Authority','Code','Period','Real_Value','Real_Numerator','Real_Denominator','Cash_Value','Cash_Numerator','Cash_Denominator','ScotRank','ScotPct']]
all_LGBFData


,GSS Code,Local_Authority,Code,Period,Real_Value,Real_Numerator,Real_Denominator,Cash_Value,Cash_Numerator,Cash_Denominator,ScotRank,ScotPct
0,S12000033,Aberdeen City,CHN1,2010-11,6193.572980,7.370352e+07,11900.0,4960.336134,59028000.0,11900.0,22.0,0.68750
1,S12000034,Aberdeenshire,CHN1,2010-11,6050.771574,1.156484e+08,19113.0,4845.968712,92621000.0,19113.0,20.0,0.62500
2,S12000041,Angus,CHN1,2010-11,5838.823993,4.928551e+07,8441.0,4676.223196,39472000.0,8441.0,11.0,0.34375
3,S12000035,Argyll & Bute,CHN1,2010-11,7302.747865,4.241436e+07,5808.0,5848.657025,33969000.0,5808.0,29.0,0.90625
4,S12000005,Clackmannanshire,CHN1,2010-11,5521.957869,2.100553e+07,3804.0,4422.450053,16823000.0,3804.0,4.0,0.12500
...,...,...,...,...,...,...,...,...,...,...,...,...
28471,S12000028,South Ayrshire,CLIM2,2020-21,4.925712,NaN,NaN,4.925712,NaN,NaN,20.0,0.62500
28472,S12000029,South Lanarkshire,CLIM2,2020-21,3.338181,NaN,NaN,3.338181,NaN,NaN,5.0,0.15625
28473,S12000030,Stirling,CLIM2,2020-21,5.349171,NaN,NaN,5.349171,NaN,NaN,28.0,0.87500
28474,S12000039,West Dunbartonshire,CLIM2,2020-21,3.611371,NaN,NaN,3.611371,NaN,NaN,7.0,0.21875


### 7.2 Family Group Ranks & Percentiles

In [23]:
#Merge all_LGBFData with additional data from indicatorInformation and Family_Groups dataframes so that the dataframe contains all of the fields required to group for the purposes of ranking.
FamilyRanks_ForGroup = all_LGBFData.merge(indicatorInformation[['Code','FamilyGrouping']], how = 'left', on = ['Code'], suffixes = ('_all','_info'))
FamilyRanks_ForGroup = FamilyRanks_ForGroup.rename(columns = {'FamilyGrouping' : 'Type'})
FamilyRanks_ForGroup = FamilyRanks_ForGroup.merge(Family_Groups,how ='left', on = ['Local_Authority','Type'], suffixes = ('_all','_group'))

#Copy the dataframe to a new variable. The FamilyRanks_ForGroup dataframe will be used again later to calculate goldilocks ranks and quartiles.
FamilyRanks = FamilyRanks_ForGroup.copy(deep = True)

#Add columns for both ascending and descending ranks and percentiles. The correct versions will be chosen later.
FamilyRanks['FamilyRank_Desc'] = FamilyRanks_ForGroup.groupby(['Code','Period','Family_Group'])['Real_Value'].rank('min',ascending = False).astype(int)
FamilyRanks['FamilyRank_Asc'] = FamilyRanks_ForGroup.groupby(['Code','Period','Family_Group'])['Real_Value'].rank('min',ascending = True).astype(int)
FamilyRanks['FamilyRank_Desc_Pct'] = FamilyRanks_ForGroup.groupby(['Code','Period','Family_Group'])['Real_Value'].rank('min',ascending = False, pct = True).astype(float)
FamilyRanks['FamilyRank_Asc_Pct'] = FamilyRanks_ForGroup.groupby(['Code','Period','Family_Group'])['Real_Value'].rank('min',ascending = True, pct = True).astype(float)

#Merge FamilyRanks_ForGroup with the midpoint for goldilocks ranks into a new dataframe
GoldilocksFamilyRanks = FamilyRanks_ForGroup.merge(indicatorInformation[['Code','Ranking_GoldilocksMidpoint']], how = 'left', on = ['Code'], suffixes = ('_Goldi','_Info'))

#Calculate ranking and percentile based on distance from goldilocks mid point.
GoldilocksFamilyRanks['AbsoluteDifferenceFromGoldilocksMidPoint'] = GoldilocksFamilyRanks.apply(DifferenceFromGoldilocksMidPoint, axis = 1)
GoldilocksFamilyRanks = GoldilocksFamilyRanks[pd.notnull(GoldilocksFamilyRanks['AbsoluteDifferenceFromGoldilocksMidPoint'])]
GoldilocksFamilyRanks['FamilyRank_Goldi'] = GoldilocksFamilyRanks.groupby(['Code','Period','Family_Group'])['AbsoluteDifferenceFromGoldilocksMidPoint'].rank('min',ascending = True).astype(int)
GoldilocksFamilyRanks['FamilyRank_Goldi_Pct'] = GoldilocksFamilyRanks.groupby(['Code','Period', 'Family_Group'])['AbsoluteDifferenceFromGoldilocksMidPoint'].rank('min',ascending = True,pct = True).astype(float)


#Merge relevant columns from GoldilocksFamilyRanks into the main FamilyRanks dataframe.
FamilyRanks = FamilyRanks.merge(GoldilocksFamilyRanks[['Code','Period','Local_Authority','FamilyRank_Goldi','FamilyRank_Goldi_Pct']], how = 'left', on = ['Code','Period','Local_Authority'], suffixes = ('_FamilyRank','_Goldi'))

#Merge the ranking type from the indicatorInformation dataframe. This will allow us to select the correct ranking and percentile in the next steps.
FamilyRanks = FamilyRanks.merge(indicatorInformation[['Code','Ranking_Type']], how = 'left', on = ['Code'], suffixes = ('_FamilyAv','_Info'))


# Define functions needed to select correct ranking type and percentile type
def FamilyRank_select(df) : 
    if df['Ranking_Type'] == "Ascending" :
        return df['FamilyRank_Asc']
    elif df['Ranking_Type'] == "Descending" :
        return df['FamilyRank_Desc']
    elif df['Ranking_Type'] == "Goldilocks" :
        return df['FamilyRank_Goldi']
    else :
        return None

def FamilyRank_Pct_select(df) : 
    if df['Ranking_Type'] == "Ascending" :
        return df['FamilyRank_Asc_Pct']
    elif df['Ranking_Type'] == "Descending" :
        return df['FamilyRank_Desc_Pct']
    elif df['Ranking_Type'] == "Goldilocks" :
        return df['FamilyRank_Goldi_Pct']
    else :
        return None

# Apply functions above to create two new columns that contain the correct rank and percentile for each row
FamilyRanks['FamilyRank'] = FamilyRanks.apply(FamilyRank_select, axis = 1)
FamilyRanks['FamilyPct'] = FamilyRanks.apply(FamilyRank_Pct_select, axis = 1)

FamilyRanks = FamilyRanks[['GSS Code','Local_Authority','Code','Period','FamilyRank','FamilyPct']]

# Merge the relevant columns from FamilyRanks into the main all_LGBFData dataframe
all_LGBFData = all_LGBFData.merge(FamilyRanks,how = 'left', on = ['GSS Code','Local_Authority','Code','Period'], suffixes = ('_FamilyAv','_Info'))

all_LGBFData


,GSS Code,Local_Authority,Code,Period,Real_Value,Real_Numerator,Real_Denominator,Cash_Value,Cash_Numerator,Cash_Denominator,ScotRank,ScotPct,FamilyRank,FamilyPct
0,S12000033,Aberdeen City,CHN1,2010-11,6193.572980,7.370352e+07,11900.0,4960.336134,59028000.0,11900.0,22.0,0.68750,6.0,0.750
1,S12000034,Aberdeenshire,CHN1,2010-11,6050.771574,1.156484e+08,19113.0,4845.968712,92621000.0,19113.0,20.0,0.62500,5.0,0.625
2,S12000041,Angus,CHN1,2010-11,5838.823993,4.928551e+07,8441.0,4676.223196,39472000.0,8441.0,11.0,0.34375,3.0,0.375
3,S12000035,Argyll & Bute,CHN1,2010-11,7302.747865,4.241436e+07,5808.0,5848.657025,33969000.0,5808.0,29.0,0.90625,8.0,1.000
4,S12000005,Clackmannanshire,CHN1,2010-11,5521.957869,2.100553e+07,3804.0,4422.450053,16823000.0,3804.0,4.0,0.12500,3.0,0.375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28471,S12000028,South Ayrshire,CLIM2,2020-21,4.925712,NaN,NaN,4.925712,NaN,NaN,20.0,0.62500,4.0,0.500
28472,S12000029,South Lanarkshire,CLIM2,2020-21,3.338181,NaN,NaN,3.338181,NaN,NaN,5.0,0.15625,2.0,0.250
28473,S12000030,Stirling,CLIM2,2020-21,5.349171,NaN,NaN,5.349171,NaN,NaN,28.0,0.87500,7.0,0.875
28474,S12000039,West Dunbartonshire,CLIM2,2020-21,3.611371,NaN,NaN,3.611371,NaN,NaN,7.0,0.21875,4.0,0.500


## 8 Adding Previous Period and Initial Row Data & Comparisons

### 8.1 Add previous and inital rows as dictionaries in their own columns

In [24]:
#Sort rows of the all_LGBFData dataframe to ensure that we are getting the correct previous and first rows in the subsequent steps. This may not be strictly necessary as the data should already be in the correct sort order. It is here as a safeguard in case any sorting needs to be done in previous steps at a later date.
all_LGBFData.sort_values(by = ['Local_Authority','Code','Period'],inplace = True)

#Define all variables that will be used to record the changes from previous/first.
Previouss = []
Previous = None
Firsts = []
First = None
First_Save = None
Local_Authority = ""
Code = ""
Period = ""
Real_Value = ""
Real_Numerator = ""
Real_Denominator = ""
Cash_Value = ""
Cash_Numerator = ""
Cash_Denominator = ""
ScotRank = ""
ScotPct = ""
FamilyRank = ""
FamilyPct = ""

# Loop over the all_LGBFDataframe and record previous and first into a python dictionary object. Save the objects for each row into the two list variables (Previouss and Firsts)
for row in all_LGBFData.itertuples() :
    # If the curently stored Local_Authority and Code are both equal to the current row then this is not the first row for this indicator and local authority combination. As such Previous is calculated using all of the currently stored values in the variables (these are written to at the end of each loop) and First is populated using the stored dictionary in First_Save
    if Local_Authority == row.Local_Authority and Code == row.Code :
        Previous = {
            'Real_Value' : Real_Value,
            'Real_Numerator' : Real_Numerator,
            'Real_Denominator' : Real_Denominator,
            'Cash_Value' : Cash_Value,
            'Cash_Numerator' : Cash_Numerator,
            'Cash_Denominator' : Cash_Denominator,
            'ScotRank' : ScotRank,
            'ScotPct' : ScotPct,
            'FamilyRank' : FamilyRank,
            'FamilyPct' : FamilyPct
            }
        First = First_Save

    # If the curently stored Local_Authority and Code are both not equal to the current row then this is the first row for this indicator and local authority combination. as such the Previous object is set to None and the First object is populated using this rows values.
    elif Local_Authority != row.Local_Authority or Code != row.Code :
        First_Save = {
            'Real_Value' : row.Real_Value,
            'Real_Numerator' : row.Real_Numerator,
            'Real_Denominator' : row.Real_Denominator,
            'Cash_Value' : row.Cash_Value,
            'Cash_Numerator' : row.Cash_Numerator,
            'Cash_Denominator' : row.Cash_Denominator,
            'ScotRank' : row.ScotRank,
            'ScotPct' : row.ScotPct,
            'FamilyRank' : row.FamilyRank,
            'FamilyPct' : row.FamilyPct
            }
        First = None
        Previous = None
    
    # Append the First and Previous into their respective list variables.
    Previouss.append(Previous)
    Firsts.append(First)

    # Set all other variables to their respective columns values in the current row. This is used to both evaluate the if criteria above and to populate the next previous object.
    Local_Authority = row.Local_Authority
    Code = row.Code
    Period = row.Period
    Real_Value = row.Real_Value
    Real_Numerator = row.Real_Numerator
    Real_Denominator = row.Real_Denominator
    Cash_Value = row.Cash_Value
    Cash_Numerator = row.Cash_Numerator
    Cash_Denominator = row.Cash_Denominator
    ScotRank = row.ScotRank
    ScotPct = row.ScotPct
    FamilyRank = row.FamilyRank
    FamilyPct = row.FamilyPct
    
# Assign the two list variables into appropriately titled columns within our all_LGBFData dataframe
all_LGBFData['Previous_Row'] = Previouss
all_LGBFData['First_Row'] = Firsts

all_LGBFData

,GSS Code,Local_Authority,Code,Period,Real_Value,Real_Numerator,Real_Denominator,Cash_Value,Cash_Numerator,Cash_Denominator,ScotRank,ScotPct,FamilyRank,FamilyPct,Previous_Row,First_Row
13826,S12000033,Aberdeen City,C&L1,2010-11,0.412044,7.953707e+05,1922292.000,0.330000,637000.0,1922292.000,1.0,0.03125,1.0,0.125,None,None
13858,S12000033,Aberdeen City,C&L1,2011-12,0.934687,1.911482e+06,2045051.000,0.759883,1554000.0,2045051.000,1.0,0.03125,1.0,0.125,"{'Real_Value': 0.41204447198001937, 'Real_Nume...","{'Real_Value': 0.41204447198001937, 'Real_Nume..."
13890,S12000033,Aberdeen City,C&L1,2012-13,4.392708,9.504749e+06,2163756.000,3.643202,7883000.0,2163756.000,24.0,0.75000,5.0,0.625,"{'Real_Value': 0.9346865726552549, 'Real_Numer...","{'Real_Value': 0.41204447198001937, 'Real_Nume..."
13922,S12000033,Aberdeen City,C&L1,2013-14,4.097409,9.106851e+06,2222588.000,3.476128,7726000.0,2222588.000,23.0,0.71875,5.0,0.625,"{'Real_Value': 4.392708423720768, 'Real_Numera...","{'Real_Value': 0.41204447198001937, 'Real_Nume..."
13954,S12000033,Aberdeen City,C&L1,2014-15,3.521952,8.759581e+06,2487138.000,3.022349,7517000.0,2487138.000,20.0,0.62500,5.0,0.625,"{'Real_Value': 4.097408554145169, 'Real_Numera...","{'Real_Value': 0.41204447198001937, 'Real_Nume..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13697,S12000040,West Lothian,SW8,2016-17,822.281625,9.536000e+03,11.597,822.281625,9536.0,11.597,20.0,0.62500,5.0,0.625,"{'Real_Value': 485.21026372059873, 'Real_Numer...","{'Real_Value': 548.7757856178908, 'Real_Numera..."
13729,S12000040,West Lothian,SW8,2017-18,1139.053501,1.364700e+04,11.981,1139.053501,13647.0,11.981,28.0,0.87500,8.0,1.000,"{'Real_Value': 822.2816245580755, 'Real_Numera...","{'Real_Value': 548.7757856178908, 'Real_Numera..."
13761,S12000040,West Lothian,SW8,2018-19,1214.250000,1.506400e+04,12.406,1214.250000,15064.0,12.406,27.0,0.84375,7.0,0.875,"{'Real_Value': 1139.0535013771805, 'Real_Numer...","{'Real_Value': 548.7757856178908, 'Real_Numera..."
13793,S12000040,West Lothian,SW8,2019-20,934.422385,1.195500e+04,12.794,934.422385,11955.0,12.794,23.0,0.71875,5.0,0.625,"{'Real_Value': 1214.25, 'Real_Numerator': 1506...","{'Real_Value': 548.7757856178908, 'Real_Numera..."


### 8.2 Add comparisons to previous and first data

In [25]:
# Define a function that will return an aim adjusted percentage change between two indicator values. There are two niche cases here. One where previous and current values are both 0 resulting in 0% in all cases. Another where only the previous value is 0 resulting in None being returned as it is not possible to calculate % change from 0. Having looked at the dataset this has only occured 3 times and only affects Orkney and Eilean Siar for CHN20b. Further to this changes in percentage indicators are calculated using 100 as a denominator rather than previous. This is to avoid situations where very small percentages return 1000% or more change (which for our purposes seemed unreasonable to report).
def PercentChange_AimAdjusted (Previous,Current,Code) :
    Aim  = None
    SignedChange = None
    PercentChange = None
    GoldiMid = None
    IsPercentage = False

    if Previous == 0 and Current == 0 :
        PercentChange = 0
    
    indicatorInfo = indicatorInformation_dict[Code]
    Aim = indicatorInfo['Ranking_Type']
    GoldiMid = indicatorInfo['Ranking_GoldilocksMidpoint']
    IsPercentage = indicatorInfo['MeasureType'] == "Percentage"

    if IsPercentage == False and Previous != 0 :
        if Aim == "Descending": 
            SignedChange = distance(Current,Previous)
            PercentChange = SignedChange/Previous
        if Aim == "Ascending":
            SignedChange = -distance(Current,Previous)
            PercentChange = SignedChange/Previous
        if Aim == "Goldilocks":
            Current_DistGoldi = abs(distance(Current,GoldiMid))
            Previous_DistGoldi = abs(distance(Previous,GoldiMid))
            SignedChange = distance(Previous_DistGoldi,Current_DistGoldi)
            PercentChange = SignedChange/Previous_DistGoldi

    elif IsPercentage == True and Previous != 0 :
        if Aim == "Descending": 
            SignedChange = distance(Current,Previous)
            PercentChange = SignedChange/100
        if Aim == "Ascending":
            SignedChange = -distance(Current,Previous)
            PercentChange = SignedChange/100
        if Aim == "Goldilocks":
            Current_DistGoldi = abs(distance(Current,GoldiMid))
            Previous_DistGoldi = abs(distance(Previous,GoldiMid))
            SignedChange = distance(Previous_DistGoldi,Current_DistGoldi)
            PercentChange = SignedChange/100

    return PercentChange

# Define a function that calculates the changes in the current row by accessing values in the Previous_Row and First_Row python dictionaries.
def Changes(df) :
    
    # Set the intial value of the return variable to none. This allows us to test to see if there were any changes present for a row and then return None instead of a dictionary of None values if not.
    Changes = None

    # Define all variables that will contain all of the relevant changes for a row.
    ScotRank_ChangeSincePrevious = None
    ScotPct_ChangeSincePrevious = None
    FamilyRank_ChangeSincePrevious = None
    FamilyPct_ChangeSincePrevious = None
    Real_Value_ChangeSincePrevious = None
    Real_Numerator_ChangeSincePrevious = None
    Real_Denominator_ChangeSincePrevious = None
    Cash_Value_ChangeSincePrevious = None
    Cash_Numerator_ChangeSincePrevious = None
    Cash_Denominator_ChangeSincePrevious = None
    ScotRank_ChangeSinceFirst = None
    ScotPct_ChangeSinceFirst = None
    FamilyRank_ChangeSinceFirst = None
    FamilyPct_ChangeSinceFirst = None
    Real_Value_ChangeSinceFirst = None
    Real_Numerator_ChangeSinceFirst = None
    Real_Denominator_ChangeSinceFirst = None
    Cash_Value_ChangeSinceFirst = None
    Cash_Numerator_ChangeSinceFirst = None
    Cash_Denominator_ChangeSinceFirst = None
    PercentChange_AimAdjusted_SincePrevious = None
    PercentChange_AimAdjusted_SinceFirst = None

    # If the value currently in Previous_Row is not None then there exists a previous object to calculate the changes using.
    if df['Previous_Row'] != None :
        # Calculate all differences by comparing the current rows value to the same columns value in the Previous_Row dictionary. Ranks and Percentiles are always positive so the calculations are more simple. The other values use the distance function defined at the start of the notebook to determine the signed difference between the values (comparing a current value of -1 to a previous value of 2 will result in -3 difference.)
        ScotRank_ChangeSincePrevious = - (df['ScotRank'] - df['Previous_Row'].get('ScotRank'))
        ScotPct_ChangeSincePrevious = - (df['ScotPct'] - df['Previous_Row'].get('ScotPct'))
        FamilyRank_ChangeSincePrevious = - (df['FamilyRank'] - df['Previous_Row'].get('FamilyRank'))
        FamilyPct_ChangeSincePrevious = - (df['FamilyPct'] - df['Previous_Row'].get('FamilyPct'))
        Real_Value_ChangeSincePrevious = distance(df['Real_Value'],df['Previous_Row'].get('Real_Value'))
        Real_Numerator_ChangeSincePrevious = distance(df['Real_Numerator'],df['Previous_Row'].get('Real_Numerator'))
        Real_Denominator_ChangeSincePrevious = distance(df['Real_Denominator'],df['Previous_Row'].get('Real_Denominator'))
        Cash_Value_ChangeSincePrevious = distance(df['Cash_Value'],df['Previous_Row'].get('Cash_Value'))
        Cash_Numerator_ChangeSincePrevious = distance(df['Cash_Numerator'],df['Previous_Row'].get('Cash_Numerator'))
        Cash_Denominator_ChangeSincePrevious = distance(df['Cash_Denominator'],df['Previous_Row'].get('Cash_Denominator'))
        PercentChange_AimAdjusted_SincePrevious = PercentChange_AimAdjusted(df['Previous_Row'].get('Real_Value'),df['Real_Value'],df['Code'])
        #Set Changes to true to avoid creating a dictionary of None values
        Changes = True
    
    # If the value currently in First_Row is not None then there exists a previous object to calculate the changes using.
    if df['First_Row'] != None :
                # Calculate all differences by comparing the current rows value to the same columns value in the First_Row dictionary. Ranks and Percentiles are always positive so the calculations are more simple. The other values use the distance function defined at the start of the notebook to determine the signed difference between the values (comparing a current value of -1 to a previous value of 2 will result in -3 difference.)
        ScotRank_ChangeSinceFirst = - (df['ScotRank'] - df['First_Row'].get('ScotRank'))
        ScotPct_ChangeSinceFirst = - (df['ScotPct'] - df['First_Row'].get('ScotPct'))
        FamilyRank_ChangeSinceFirst = - (df['FamilyRank'] - df['First_Row'].get('FamilyRank'))
        FamilyPct_ChangeSinceFirst = - (df['FamilyPct'] - df['First_Row'].get('FamilyPct'))
        Real_Value_ChangeSinceFirst = distance(df['Real_Value'],df['First_Row'].get('Real_Value'))
        Real_Numerator_ChangeSinceFirst = distance(df['Real_Numerator'],df['First_Row'].get('Real_Numerator'))
        Real_Denominator_ChangeSinceFirst = distance(df['Real_Denominator'],df['First_Row'].get('Real_Denominator'))
        Cash_Value_ChangeSinceFirst = distance(df['Cash_Value'],df['First_Row'].get('Cash_Value'))
        Cash_Numerator_ChangeSinceFirst = distance(df['Cash_Numerator'],df['First_Row'].get('Cash_Numerator'))
        Cash_Denominator_ChangeSinceFirst = distance(df['Cash_Denominator'],df['First_Row'].get('Cash_Denominator'))
        PercentChange_AimAdjusted_SinceFirst = PercentChange_AimAdjusted(df['First_Row'].get('Real_Value'),df['Real_Value'],df['Code'])
        #Set Changes to true to avoid creating a dictionary of None values
        Changes = True

    # If there were changes recorded in the previous steps then write these changes into a python dictionary and assign this to Changes
    if Changes != None :
        Changes = {
            "ScotRank_ChangeSincePrevious" : ScotRank_ChangeSincePrevious,
            "ScotPct_ChangeSincePrevious" : ScotPct_ChangeSincePrevious,
            "FamilyRank_ChangeSincePrevious" : FamilyRank_ChangeSincePrevious,
            "FamilyPct_ChangeSincePrevious" : FamilyPct_ChangeSincePrevious,
            "ScotRank_ChangeSinceFirst" : ScotRank_ChangeSinceFirst,
            "ScotPct_ChangeSinceFirst" : ScotPct_ChangeSinceFirst,
            "FamilyRank_ChangeSinceFirst" : FamilyRank_ChangeSinceFirst,
            "FamilyPct_ChangeSinceFirst" : FamilyPct_ChangeSinceFirst,
            "Real_Value_ChangeSincePrevious" : Real_Value_ChangeSincePrevious,
            "Real_Numerator_ChangeSincePrevious" : Real_Numerator_ChangeSincePrevious,
            "Real_Denominator_ChangeSincePrevious" : Real_Denominator_ChangeSincePrevious,
            "Cash_Value_ChangeSincePrevious" : Cash_Value_ChangeSincePrevious,
            "Cash_Numerator_ChangeSincePrevious" : Cash_Numerator_ChangeSincePrevious,
            "Cash_Denominator_ChangeSincePrevious" : Cash_Denominator_ChangeSincePrevious,
            "Real_Value_ChangeSinceFirst" : Real_Value_ChangeSinceFirst,
            "Real_Numerator_ChangeSinceFirst" : Real_Numerator_ChangeSinceFirst,
            "Real_Denominator_ChangeSinceFirst" : Real_Denominator_ChangeSinceFirst,
            "Cash_Value_ChangeSinceFirst" : Cash_Value_ChangeSinceFirst,
            "Cash_Numerator_ChangeSinceFirst" : Cash_Numerator_ChangeSinceFirst,
            "Cash_Denominator_ChangeSinceFirst" : Cash_Denominator_ChangeSinceFirst,
            "PercentChange_AimAdjusted_SincePrevious" : PercentChange_AimAdjusted_SincePrevious,
            "PercentChange_AimAdjusted_SinceFirst" : PercentChange_AimAdjusted_SinceFirst
        }

    return Changes

#Apply the above function and output it to a new column named Changes.
all_LGBFData['Changes'] = all_LGBFData.apply(Changes, axis = 1)
all_LGBFData

,GSS Code,Local_Authority,Code,Period,Real_Value,Real_Numerator,Real_Denominator,Cash_Value,Cash_Numerator,Cash_Denominator,ScotRank,ScotPct,FamilyRank,FamilyPct,Previous_Row,First_Row,Changes
13826,S12000033,Aberdeen City,C&L1,2010-11,0.412044,7.953707e+05,1922292.000,0.330000,637000.0,1922292.000,1.0,0.03125,1.0,0.125,None,None,None
13858,S12000033,Aberdeen City,C&L1,2011-12,0.934687,1.911482e+06,2045051.000,0.759883,1554000.0,2045051.000,1.0,0.03125,1.0,0.125,"{'Real_Value': 0.41204447198001937, 'Real_Nume...","{'Real_Value': 0.41204447198001937, 'Real_Nume...","{'ScotRank_ChangeSincePrevious': -0.0, 'ScotPc..."
13890,S12000033,Aberdeen City,C&L1,2012-13,4.392708,9.504749e+06,2163756.000,3.643202,7883000.0,2163756.000,24.0,0.75000,5.0,0.625,"{'Real_Value': 0.9346865726552549, 'Real_Numer...","{'Real_Value': 0.41204447198001937, 'Real_Nume...","{'ScotRank_ChangeSincePrevious': -23.0, 'ScotP..."
13922,S12000033,Aberdeen City,C&L1,2013-14,4.097409,9.106851e+06,2222588.000,3.476128,7726000.0,2222588.000,23.0,0.71875,5.0,0.625,"{'Real_Value': 4.392708423720768, 'Real_Numera...","{'Real_Value': 0.41204447198001937, 'Real_Nume...","{'ScotRank_ChangeSincePrevious': 1.0, 'ScotPct..."
13954,S12000033,Aberdeen City,C&L1,2014-15,3.521952,8.759581e+06,2487138.000,3.022349,7517000.0,2487138.000,20.0,0.62500,5.0,0.625,"{'Real_Value': 4.097408554145169, 'Real_Numera...","{'Real_Value': 0.41204447198001937, 'Real_Nume...","{'ScotRank_ChangeSincePrevious': 3.0, 'ScotPct..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13697,S12000040,West Lothian,SW8,2016-17,822.281625,9.536000e+03,11.597,822.281625,9536.0,11.597,20.0,0.62500,5.0,0.625,"{'Real_Value': 485.21026372059873, 'Real_Numer...","{'Real_Value': 548.7757856178908, 'Real_Numera...","{'ScotRank_ChangeSincePrevious': -10.0, 'ScotP..."
13729,S12000040,West Lothian,SW8,2017-18,1139.053501,1.364700e+04,11.981,1139.053501,13647.0,11.981,28.0,0.87500,8.0,1.000,"{'Real_Value': 822.2816245580755, 'Real_Numera...","{'Real_Value': 548.7757856178908, 'Real_Numera...","{'ScotRank_ChangeSincePrevious': -8.0, 'ScotPc..."
13761,S12000040,West Lothian,SW8,2018-19,1214.250000,1.506400e+04,12.406,1214.250000,15064.0,12.406,27.0,0.84375,7.0,0.875,"{'Real_Value': 1139.0535013771805, 'Real_Numer...","{'Real_Value': 548.7757856178908, 'Real_Numera...","{'ScotRank_ChangeSincePrevious': 1.0, 'ScotPct..."
13793,S12000040,West Lothian,SW8,2019-20,934.422385,1.195500e+04,12.794,934.422385,11955.0,12.794,23.0,0.71875,5.0,0.625,"{'Real_Value': 1214.25, 'Real_Numerator': 1506...","{'Real_Value': 548.7757856178908, 'Real_Numera...","{'ScotRank_ChangeSincePrevious': 4.0, 'ScotPct..."


### 8.3 Convert Python Dictionaries to JSON
This is to make the final file more usable in BI products such as PowerBI

In [26]:
def PreviousConvertToJson (df) :
    Previous_Row = simplejson.dumps(df['Previous_Row'],ignore_nan=True)    
    return Previous_Row

def FirstConvertToJson (df) :
    First_Row = simplejson.dumps(df['First_Row'],ignore_nan=True)    
    return First_Row

def ChangesConvertToJson (df) :
    Changes = simplejson.dumps(df['Changes'],ignore_nan=True)    
    return Changes

all_LGBFData['Previous_Row'] = all_LGBFData.apply(PreviousConvertToJson, axis = 1)
all_LGBFData['First_Row'] = all_LGBFData.apply(FirstConvertToJson, axis = 1)
all_LGBFData['Changes'] = all_LGBFData.apply(ChangesConvertToJson, axis = 1)

# Merge sortable codes into dataframe
all_LGBFData = all_LGBFData.merge(indicatorInformation[['Code','Code_Sortable']], how = 'left', on = ['Code'], suffixes = ('ISScot','_info'))

# Add relationship Key Columns
all_LGBFData['Key_CodePeriod'] = all_LGBFData['Code_Sortable'] + all_LGBFData['Period']
all_LGBFData['Key_CodePeriodLA'] = all_LGBFData['Code_Sortable'] + all_LGBFData['Period'] + all_LGBFData['Local_Authority']
all_LGBFData = all_LGBFData.merge(indicatorInformation[['Code_Sortable','FamilyGrouping']], how = 'left', on = ['Code_Sortable'], suffixes = ('_all','_info'))
all_LGBFData = all_LGBFData.rename(columns = {'FamilyGrouping' : 'Type'})
all_LGBFData = all_LGBFData.merge(Family_Groups,how ='left', on = ['Local_Authority','Type'], suffixes = ('_all','_group'))
all_LGBFData['Key_CodePeriodFamilyGroup'] = all_LGBFData['Code_Sortable'] + all_LGBFData['Period'] + all_LGBFData['Family_Group']

# Reorder and rename columns appropriately
all_LGBFData = all_LGBFData[['Key_CodePeriod','Key_CodePeriodFamilyGroup','Key_CodePeriodLA','GSS Code','Local_Authority','Code_Sortable','Period','Real_Value','Real_Numerator','Real_Denominator','Cash_Value','Cash_Numerator','Cash_Denominator','ScotRank','ScotPct','FamilyRank','FamilyPct','Previous_Row','First_Row','Changes']]
all_LGBFData = all_LGBFData.rename(columns = {'Code_Sortable' : 'Code'})

all_LGBFData

,Key_CodePeriod,Key_CodePeriodFamilyGroup,Key_CodePeriodLA,GSS Code,Local_Authority,Code,Period,Real_Value,Real_Numerator,Real_Denominator,Cash_Value,Cash_Numerator,Cash_Denominator,ScotRank,ScotPct,FamilyRank,FamilyPct,Previous_Row,First_Row,Changes
0,C&L 012010-11,C&L 012010-11Family Group 4,C&L 012010-11Aberdeen City,S12000033,Aberdeen City,C&L 01,2010-11,0.412044,7.953707e+05,1922292.000,0.330000,637000.0,1922292.000,1.0,0.03125,1.0,0.125,null,null,null
1,C&L 012011-12,C&L 012011-12Family Group 4,C&L 012011-12Aberdeen City,S12000033,Aberdeen City,C&L 01,2011-12,0.934687,1.911482e+06,2045051.000,0.759883,1554000.0,2045051.000,1.0,0.03125,1.0,0.125,"{""Real_Value"": 0.41204447198001937, ""Real_Nume...","{""Real_Value"": 0.41204447198001937, ""Real_Nume...","{""ScotRank_ChangeSincePrevious"": -0.0, ""ScotPc..."
2,C&L 012012-13,C&L 012012-13Family Group 4,C&L 012012-13Aberdeen City,S12000033,Aberdeen City,C&L 01,2012-13,4.392708,9.504749e+06,2163756.000,3.643202,7883000.0,2163756.000,24.0,0.75000,5.0,0.625,"{""Real_Value"": 0.9346865726552549, ""Real_Numer...","{""Real_Value"": 0.41204447198001937, ""Real_Nume...","{""ScotRank_ChangeSincePrevious"": -23.0, ""ScotP..."
3,C&L 012013-14,C&L 012013-14Family Group 4,C&L 012013-14Aberdeen City,S12000033,Aberdeen City,C&L 01,2013-14,4.097409,9.106851e+06,2222588.000,3.476128,7726000.0,2222588.000,23.0,0.71875,5.0,0.625,"{""Real_Value"": 4.392708423720768, ""Real_Numera...","{""Real_Value"": 0.41204447198001937, ""Real_Nume...","{""ScotRank_ChangeSincePrevious"": 1.0, ""ScotPct..."
4,C&L 012014-15,C&L 012014-15Family Group 4,C&L 012014-15Aberdeen City,S12000033,Aberdeen City,C&L 01,2014-15,3.521952,8.759581e+06,2487138.000,3.022349,7517000.0,2487138.000,20.0,0.62500,5.0,0.625,"{""Real_Value"": 4.097408554145169, ""Real_Numera...","{""Real_Value"": 0.41204447198001937, ""Real_Nume...","{""ScotRank_ChangeSincePrevious"": 3.0, ""ScotPct..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28471,SW 082016-17,SW 082016-17Family Group 3,SW 082016-17West Lothian,S12000040,West Lothian,SW 08,2016-17,822.281625,9.536000e+03,11.597,822.281625,9536.0,11.597,20.0,0.62500,5.0,0.625,"{""Real_Value"": 485.21026372059873, ""Real_Numer...","{""Real_Value"": 548.7757856178908, ""Real_Numera...","{""ScotRank_ChangeSincePrevious"": -10.0, ""ScotP..."
28472,SW 082017-18,SW 082017-18Family Group 3,SW 082017-18West Lothian,S12000040,West Lothian,SW 08,2017-18,1139.053501,1.364700e+04,11.981,1139.053501,13647.0,11.981,28.0,0.87500,8.0,1.000,"{""Real_Value"": 822.2816245580755, ""Real_Numera...","{""Real_Value"": 548.7757856178908, ""Real_Numera...","{""ScotRank_ChangeSincePrevious"": -8.0, ""ScotPc..."
28473,SW 082018-19,SW 082018-19Family Group 3,SW 082018-19West Lothian,S12000040,West Lothian,SW 08,2018-19,1214.250000,1.506400e+04,12.406,1214.250000,15064.0,12.406,27.0,0.84375,7.0,0.875,"{""Real_Value"": 1139.0535013771805, ""Real_Numer...","{""Real_Value"": 548.7757856178908, ""Real_Numera...","{""ScotRank_ChangeSincePrevious"": 1.0, ""ScotPct..."
28474,SW 082019-20,SW 082019-20Family Group 3,SW 082019-20West Lothian,S12000040,West Lothian,SW 08,2019-20,934.422385,1.195500e+04,12.794,934.422385,11955.0,12.794,23.0,0.71875,5.0,0.625,"{""Real_Value"": 1214.25, ""Real_Numerator"": 1506...","{""Real_Value"": 548.7757856178908, ""Real_Numera...","{""ScotRank_ChangeSincePrevious"": 4.0, ""ScotPct..."


## 9 Split Last Values Into Separate Dataframe

In [27]:
LastValues = all_LGBFData.copy(deep = True)
LastValues.sort_values(by = ['Code','Period','Local_Authority'],inplace = True)
LastValues = LastValues.groupby(['Code','Local_Authority']).tail(1)
LastValues['Key_CodePeriodLA'] = LastValues['Code'] + LastValues['Period'] + LastValues['Local_Authority']
LastValues = LastValues[['Key_CodePeriodLA','GSS Code','Local_Authority','Code','Period','Real_Value','Real_Numerator','Real_Denominator','Cash_Value','Cash_Numerator','Cash_Denominator','ScotRank','ScotPct','FamilyRank','FamilyPct','Previous_Row','First_Row','Changes']]
LastValues

,Key_CodePeriodLA,GSS Code,Local_Authority,Code,Period,Real_Value,Real_Numerator,Real_Denominator,Cash_Value,Cash_Numerator,Cash_Denominator,ScotRank,ScotPct,FamilyRank,FamilyPct,Previous_Row,First_Row,Changes
10,C&L 012020-21Aberdeen City,S12000033,Aberdeen City,C&L 01,2020-21,27.680680,7032000.0,254040.000,27.680680,7032000.0,254040.000,13.0,0.40625,3.0,0.375,"{""Real_Value"": 3.6796210540714442, ""Real_Numer...","{""Real_Value"": 0.41204447198001937, ""Real_Nume...","{""ScotRank_ChangeSincePrevious"": 11.0, ""ScotPc..."
917,C&L 012020-21Aberdeenshire,S12000034,Aberdeenshire,C&L 01,2020-21,30.023715,8381000.0,279146.000,30.023715,8381000.0,279146.000,17.0,0.53125,7.0,0.875,"{""Real_Value"": 3.179159518885439, ""Real_Numera...","{""Real_Value"": 5.5938158620317795, ""Real_Numer...","{""ScotRank_ChangeSincePrevious"": 4.0, ""ScotPct..."
1820,C&L 012020-21Angus,S12000041,Angus,C&L 01,2020-21,113.108185,2964000.0,26205.000,113.108185,2964000.0,26205.000,30.0,0.93750,8.0,1.000,"{""Real_Value"": 3.0614571174633043, ""Real_Numer...","{""Real_Value"": 2.447294439638903, ""Real_Numera...","{""ScotRank_ChangeSincePrevious"": -10.0, ""ScotP..."
2728,C&L 012020-21Argyll & Bute,S12000035,Argyll & Bute,C&L 01,2020-21,19.176016,1475000.0,76919.000,19.176016,1475000.0,76919.000,6.0,0.18750,3.0,0.375,"{""Real_Value"": 2.9771983996167615, ""Real_Numer...","{""Real_Value"": 3.021659461186809, ""Real_Numera...","{""ScotRank_ChangeSincePrevious"": 13.0, ""ScotPc..."
3583,C&L 012020-21Clackmannanshire,S12000005,Clackmannanshire,C&L 01,2020-21,21.030781,770000.0,36613.000,21.030781,770000.0,36613.000,7.0,0.21875,2.0,0.250,"{""Real_Value"": 0.8926976126619854, ""Real_Numer...","{""Real_Value"": 3.7458588361819944, ""Real_Numer...","{""ScotRank_ChangeSincePrevious"": -4.0, ""ScotPc..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24845,SW 082020-21South Ayrshire,S12000028,South Ayrshire,SW 08,2020-21,886.358500,11754.0,13.261,886.358500,11754.0,13.261,31.0,0.96875,8.0,1.000,"{""Real_Value"": 1699.4460884740877, ""Real_Numer...","{""Real_Value"": 629.4619982920581, ""Real_Numera...","{""ScotRank_ChangeSincePrevious"": 1.0, ""ScotPct..."
25753,SW 082020-21South Lanarkshire,S12000029,South Lanarkshire,SW 08,2020-21,544.215000,15047.0,27.649,544.215000,15047.0,27.649,20.0,0.62500,4.0,0.500,"{""Real_Value"": 1155.6851523164416, ""Real_Numer...","{""Real_Value"": 1368.7569810116481, ""Real_Numer...","{""ScotRank_ChangeSincePrevious"": 7.0, ""ScotPct..."
26660,SW 082020-21Stirling,S12000030,Stirling,SW 08,2020-21,375.073700,3181.0,8.481,375.073700,3181.0,8.481,17.0,0.53125,4.0,0.500,"{""Real_Value"": 848.7816531294792, ""Real_Numera...","{""Real_Value"": 837.0218579234972, ""Real_Numera...","{""ScotRank_ChangeSincePrevious"": 5.0, ""ScotPct..."
27567,SW 082020-21West Dunbartonshire,S12000039,West Dunbartonshire,SW 08,2020-21,909.065400,6468.0,7.115,909.065400,6468.0,7.115,32.0,1.00000,8.0,1.000,"{""Real_Value"": 541.0088025709096, ""Real_Numera...","{""Real_Value"": 705.2646930144379, ""Real_Numera...","{""ScotRank_ChangeSincePrevious"": -17.0, ""ScotP..."


## 10 Format Scottish Values

In [28]:
 # Convert value columns to numeric only using errors = 'coerce' to force any text values (dna, *, N/A etc.) to return NaN.
ISScottishValues[['Real_Value','Cash_Value']] = ISScottishValues[['Real_Value','Cash_Value']].apply(pd.to_numeric,errors='coerce')

# Remove rows which contain null or NaN in the Real_Value column as they are non-datapoints and could interfere with average, ranking and quartile calculations later.
ISScottishValues = ISScottishValues[pd.notnull(ISScottishValues['Real_Value'])]

# Convert Percentages to proper decimal percentages
ISScottishValues = ISScottishValues.merge(indicatorInformation[['Code','MeasureType']], how = 'left', on = ['Code'], suffixes = ('_all','_info'))

# These steps use the functions defined in step 5.5
ISScottishValues['Real_Value'] = ISScottishValues.apply(ConvertPercent_Real, axis = 1)
ISScottishValues['Cash_Value'] = ISScottishValues.apply(ConvertPercent_Cash, axis = 1)

ISScottishValues = ISScottishValues.drop(columns = ['MeasureType'])

# Merge sortable codes into dataframe
ISScottishValues = ISScottishValues.merge(indicatorInformation[['Code','Code_Sortable']], how = 'left', on = ['Code'], suffixes = ('ISScot','_info'))

# Add relationship Key Columns
ISScottishValues['Key_CodePeriod'] = ISScottishValues['Code_Sortable'] + ISScottishValues['Period']

# Reorder and rename columns appropriately
ISScottishValues = ISScottishValues[['Key_CodePeriod','Code_Sortable','Period','Real_Value','Cash_Value']]
ISScottishValues = ISScottishValues.rename(columns = {'Real_Value' : 'IS_Scot_Real_Value', 'Cash_Value' : 'IS_Scot_Cash_Value','Code_Sortable' : 'Code'})

ISScottishValues

,Key_CodePeriod,Code,Period,IS_Scot_Real_Value,IS_Scot_Cash_Value
0,CHN 012010-11,CHN 01,2010-11,6036.446349,4834.495863
1,CHN 012011-12,CHN 01,2011-12,5868.996585,4771.387895
2,CHN 012012-13,CHN 01,2012-13,5708.356107,4734.367021
3,CHN 012013-14,CHN 01,2013-14,5559.185492,4716.258860
4,CHN 012014-15,CHN 01,2014-15,5411.158151,4643.564279
...,...,...,...,...,...
903,CLIM 022016-17,CLIM 02,2016-17,5.176939,5.176939
904,CLIM 022017-18,CLIM 02,2017-18,5.001218,5.001218
905,CLIM 022018-19,CLIM 02,2018-19,4.837512,4.837512
906,CLIM 022019-20,CLIM 02,2019-20,4.592845,4.592845


## 11 Basic Verification Checks

In [29]:
# Default position is to assume checks are passed. If any of the checks (excluding numerator denominator checks) are failed below this will be changed and the final csv's will not be output. The numerator denominator errors should be checked at each refresh. The known errors (which exist within the LGBF raw data file) will be identified in the readme in the Error Outputs folder.
ChecksFailed = False

# ScotRank should be between 32 and 1 and should not contain any NaN values
if  not all_LGBFData['ScotRank'].between(1,32).any() or all_LGBFData['ScotRank'].isnull().values.any() :

    ChecksFailed = True
    maxrank = str(max(all_LGBFData['ScotRank']))
    minrank = str(min(all_LGBFData['ScotRank']))
    countnull = str(all_LGBFData['ScotRank'].isna().sum())

    print("ScotRank checks failed : Max - " + maxrank + ", Min - " + minrank + ", Count of Null - " + countnull)

# FamilyRank should be between 8 and 1 and should not contain any NaN values
if  not all_LGBFData['FamilyRank'].between(1,8).any() or all_LGBFData['FamilyRank'].isnull().values.any() :

    ChecksFailed = True
    maxrank = str(max(all_LGBFData['FamilyRank']))
    minrank = str(min(all_LGBFData['FamilyRank']))
    countnull = str(all_LGBFData['FamilyRank'].isna().sum())

    print("FamilyRank checks failed : Max - " + maxrank + ", Min - " + minrank + ", Count of Null - " + countnull)

# Code, Local_Authority, Period, Real_Value and Cash_Value should not contain any null values
if  all_LGBFData[['Code','Local_Authority','Period','Real_Value','Cash_Value',]].isnull().values.any() : 

    ChecksFailed = True
    countnullCode = str(all_LGBFData['Code'].isna().sum())
    countnullLocal_Authority = str(all_LGBFData['Local_Authority'].isna().sum())
    countnullPeriod = str(all_LGBFData['Period'].isna().sum())
    countnullReal_Value = str(all_LGBFData['Real_Value'].isna().sum())
    countnullCash_Value = str(all_LGBFData['Cash_Value'].isna().sum())

    print("Null values found : Code - " + countnullCode + ", Local_Authority - " + countnullLocal_Authority + ", Period - " + countnullPeriod + ", Real_Value - " + countnullReal_Value + ", Cash_Value - " + countnullCash_Value)

# Value should equal numerator/denominator for both cash and real - These errors have been checked and exist in the original raw data file.
NumDenCheck = all_LGBFData.copy(deep = True)
NumDenCheck = NumDenCheck[pd.notnull(NumDenCheck['Real_Numerator'])]

Real_NumDenDivide_Checks = []
Real_NumDenDivide_Check = None
Cash_NumDenDivide_Checks = []
Cash_NumDenDivide_Check = None
FailReferences = []

for row in NumDenCheck.itertuples() : 
    if row.Real_Value == 0 or math.isnan(row.Real_Denominator) or math.isnan(row.Real_Numerator) :
        Real_NumDenDivide_Check = None
    else :
        if math.isclose(row.Real_Numerator/row.Real_Denominator, row.Real_Value, rel_tol = 0.02): 
            Real_NumDenDivide_Check = True
        else : 
            Real_NumDenDivide_Check = False
            FailReferences.append("Real;" + row.Code + ";" + row.Period + ";" + row.Local_Authority + ";" + str(row.Real_Value) + ";" + str(row.Real_Numerator) + ";" + str(row.Real_Denominator))

        if math.isclose(row.Cash_Numerator/row.Cash_Denominator, row.Cash_Value, rel_tol = 0.02): 
            Cash_NumDenDivide_Check = True
        else : 
            Cash_NumDenDivide_Check = False
            FailReferences.append("Cash;" + row.Code + ";" + row.Period + ";" + row.Local_Authority + ";" + str(row.Cash_Value) + ";" + str(row.Cash_Numerator) + ";" + str(row.Cash_Denominator))

    Real_NumDenDivide_Checks.append(Real_NumDenDivide_Check)
    Cash_NumDenDivide_Checks.append(Cash_NumDenDivide_Check)

if False in Real_NumDenDivide_Checks or False in Cash_NumDenDivide_Checks : 
    print("Numerator/Denominator values check failed : See Error Outputs for csv of failures")
    FailReferences = sorted(list(set(FailReferences)))
    FailReferences = pd.DataFrame([sub.split(";") for sub in FailReferences])
    FailReferences = FailReferences.rename(columns = {0 : 'Type', 1 : 'Code', 2 : 'Period', 3 : 'Local Authority', 4 : 'Value', 5 : 'Numerator', 6 : 'Denominator'})
    FailReferences.to_csv("Error Outputs//Numerator Denominator Fail References.csv", index = False, encoding='utf-8-sig')

Numerator/Denominator values check failed : See Error Outputs for csv of failures


## 12 Output All Final Tables

In [30]:
if ChecksFailed == False :
    all_LGBFData.to_csv("Data Files//Indicator Data.csv", index = False, encoding='utf-8-sig')
    LastValues.to_csv("Data Files//Latest Values.csv", index = False, encoding='utf-8-sig')
    FamilyAverages.to_csv("Data Files//Family Averages.csv", index = False, encoding='utf-8-sig')
    ScottishAverages.to_csv("Data Files//Scottish Averages.csv", index = False, encoding='utf-8-sig')
    ISScottishValues.to_csv("Data Files//Scottish Values.csv", index = False, encoding='utf-8-sig')
else :
    print("Checks failed! Check output from section 11 for detail")